In [1]:
import torch
import sys
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import pysindy as ps

# Add the parent directory of the script (i.e., project/) to sys.path
sys.path.append('../../utils')
from architectures import FNN
from upinn import UPINN, NullWork
from utils import SINDy_sklearn

In [2]:
class LV_UPINN(UPINN):

    def F_input(self, z, U):
        return U

In [3]:
# Define model architectures
hidden = [16] * 4
u = FNN(
    dims=[1, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.SiLU(),
)
F = FNN(
    dims=[2, *hidden, 2],
    hidden_act=torch.nn.Tanh(),
    output_act=torch.nn.Identity(),
)

N = NullWork()

model = LV_UPINN(u, N, F)

[Info]: Initializing UPINN model


### Load model

In [4]:
path = 'models'
name = 'lv_baseline_data20_noise0000'
model.load(name, path)

In [5]:
# Generate data from the solution network
# t = data.t_full
t = torch.linspace(0, 10, 1000).reshape(-1, 1)
X = model.u(t)
X_dot = model.F(model.F_input(None, X))
t = t.detach().numpy()
X = X.detach().numpy()
X_dot = X_dot.detach().numpy()

In [6]:
# Hyperparameter tuning using Crossvalidation with TimeSeriesSplit
sindy_model = SINDy_sklearn(feature_names=['x', 'y'], feature_library=ps.PolynomialLibrary(degree=2, include_bias=False), optimizer=ps.SR3(thresholder="L0", max_iter=10000, normalize_columns=True))
paramgrid = dict(optimizer__threshold=np.linspace(0.01, 10.0, 100))

cv_folds = TimeSeriesSplit(n_splits=10)
model_selector = GridSearchCV(sindy_model, param_grid=paramgrid, cv=cv_folds, n_jobs=-1)
wrapped_input = np.concatenate([t, X], axis=1)
model_selector.fit(wrapped_input, X_dot)
best_model = model_selector.best_estimator_

best_model.print()

(x)' = 1.333 x y
(y)' = -1.000 x y
